In [5]:
import torch
from diffusers import StableDiffusionImg2ImgPipeline, DDIMScheduler
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import json
import gc

# Parameters
model_id = "stabilityai/stable-diffusion-2-1"
image_paths = [
    "../data/results/ma-boston_200250_fake_B.png",
    "../data/results/nc-charlotte_200250_fake_B.png",
    "../data/results/ny-manhattan_200250_fake_B.png",
    "../data/results/pa-pittsburgh_200250_fake_B.png"
]
num_steps = 5
inference_steps = 20
identifier = f"sd-2-1_cpu_4-images_{num_steps}-steps_{inference_steps}-inference"
output_dir = os.path.join("diffusion-output", identifier)
output_image_size = (384, 384)
max_image_dimension = 384

# Create a dictionary with all relevant parameters
params = {
    "identifier": identifier,
    "model_id": model_id,
    "num_steps": num_steps,
    "inference_steps": inference_steps,
    "output_image_size": output_image_size,
    "max_image_dimension": max_image_dimension
}

# Save parameters to JSON
os.makedirs(output_dir, exist_ok=True)
with open(os.path.join(output_dir, "parameters.json"), 'w') as f:
    json.dump(params, f, indent=4)

# Set device to CPU
device = torch.device("cpu")
print(f"Using device: {device}")

def load_and_preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = image.resize(output_image_size, Image.LANCZOS)
    return image

def slerp_images(image1, image2, alpha):
    arr1 = np.array(image1).astype(np.float32) / 255.0
    arr2 = np.array(image2).astype(np.float32) / 255.0
    
    omega = np.arccos(np.clip(np.dot(arr1.flatten(), arr2.flatten()) / 
                              (np.linalg.norm(arr1) * np.linalg.norm(arr2)), -1, 1))
    so = np.sin(omega)
    interp = np.sin((1.0-alpha)*omega) / so * arr1 + np.sin(alpha*omega) / so * arr2
    
    interp = np.clip(interp, 0, 1) * 255
    return Image.fromarray(interp.astype(np.uint8))

def clear_memory():
    gc.collect()

try:
    # Load the SD 2.1 pipeline with CPU settings
    with tqdm(total=1, desc="Loading SD 2.1 model") as pbar:
        scheduler = DDIMScheduler.from_pretrained(model_id, subfolder="scheduler")
        pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
            model_id,
            scheduler=scheduler,
            torch_dtype=torch.float32,  # Use float32 for CPU
            safety_checker=None,
            feature_extractor=None,
            requires_safety_checker=False,
        )
        pipe = pipe.to(device)
        clear_memory()
        pbar.update(1)

    # Load and preprocess images
    images = []
    with tqdm(total=len(image_paths), desc="Loading and preprocessing images") as pbar:
        for image_path in image_paths:
            image = load_and_preprocess_image(image_path)
            images.append(image)
            pbar.update(1)

    # Interpolate between images
    print("Interpolating between images...")
    alphas = np.linspace(0, 1, num_steps)
    interpolated_images = []
    
    with tqdm(total=num_steps, desc="Interpolating and Generating") as pbar:
        for alpha in alphas:
            # Interpolate between the first and last image
            interpolated_image = slerp_images(images[0], images[-1], alpha)
            
            # Generate image using SD 2.1
            with torch.no_grad():
                output = pipe(
                    prompt="A high quality image",
                    image=interpolated_image,
                    num_inference_steps=inference_steps,
                    guidance_scale=7.5,
                    strength=0.5
                ).images[0]
            
            interpolated_images.append(output)
            
            # Save the interpolated image
            output_path = os.path.join(output_dir, f"interpolated_{len(interpolated_images)}.png")
            output.save(output_path, quality=95)
            
            clear_memory()
            pbar.update(1)

    print(f"Interpolation complete. {num_steps} images generated and saved in {output_dir}")

    # Plot results and save the plot as an image file
    print("Plotting results...")
    fig, axes = plt.subplots(1, num_steps, figsize=(20, 4))
    for ax, img in zip(axes, interpolated_images):
        ax.imshow(img)
        ax.axis('off')
    
    plot_path = os.path.join(output_dir, "interpolation_steps.png")
    fig.savefig(plot_path, bbox_inches='tight')
    plt.close(fig)
    clear_memory()
    print(f"Plot saved to {plot_path}")

except Exception as e:
    print(f"An error occurred: {e}")

Using device: cuda


Loading SD 2.1 model:   0%|          | 0/1 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading and preprocessing images:   0%|          | 0/4 [00:00<?, ?it/s]

Interpolating between images...


Interpolating and Generating:   0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

An error occurred: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 7.65 GiB of which 98.94 MiB is free. Process 14794 has 256.00 MiB memory in use. Process 19628 has 3.09 GiB memory in use. Including non-PyTorch memory, this process has 2.10 GiB memory in use. Of the allocated memory 1.84 GiB is allocated by PyTorch, and 64.80 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


: 